In [ ]:
# Use tensorflow 2.5!!!
# !conda install tensorflow==2.5

## Detect from an Images

In [16]:
import os

CUSTOM_MODEL_NAME = 'my_ssd_mobilenet'

paths = {
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),

    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),

    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),

    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport')
 }

files = {
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt'),
    'TRAINING_SCRIPT': os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
}

In [17]:
# Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=1, savepath='/content/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[2]['index'])[0] # Confidence of detected objects      

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              print(ymin)
              print(xmin)
              print(ymax)
              print(xmax)

              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])


      # All the results have been drawn on the image, now display the image
      if txt_only == False: # "text_only" controls whether we want to display the image results or just save them in .txt files
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()

      # Save detection results in .txt files (for calculating mAP)
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

In [18]:
# Set up variables for running user's model
PATH_TO_IMAGES=os.path.join('Tensorflow', 'workspace','images', 'test')   # Path to test images folder
PATH_TO_MODEL='licence_model_my.tflite'   # Path to .tflite model file
PATH_TO_LABELS=os.path.join('Tensorflow', 'workspace','annotations','label_map.txt')   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 1   # Number of images to run detection on

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)

TypeError: object of type 'numpy.float32' has no len()

## Convert Saved Model to TFLite Model

In [4]:
LABELMAP_FILE = os.path.join('Tensorflow', 'workspace','annotations','label_map.txt')
# OUTPUT_MODEL_PATH = "updated_model.tflite"
OUTPUT_MODEL_PATH = "licence_model_my.tflite"
OUTPUT_MODEL_PATH_JSON = "licence_model_my.json"

In [2]:
import os
import tensorflow as tf

SAVED_MODEL_PATH = os.path.join('Tensorflow', 'workspace','models', 'my_ssd_mobilenet', 'tfliteexport', 'saved_model') 

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
tflite_model = converter.convert()

# Save the TFLite model with metadata
# tflite_model_path = os.path.join('Tensorflow', 'workspace','models', 'my_ssd_mobilenet', 'tfliteexport', 'saved_model', 'tflite_model_with_metadata.tflite') 
tflite_model_path = os.path.join(OUTPUT_MODEL_PATH) 

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

## Add Metadata to TFLite Model

In [5]:
import os
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [8]:
# Create model meta data
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
model_meta.description = ("This model detects car licence plates and is based on the ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8 model")
model_meta.version = "1.0"
model_meta.author = "Jin-Jin Lee"

In [7]:
# Create input tensor info
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified.\n"
    "One input: image, a float32 tensor of shape[1, height, width, 3] containing "
    "the *normalized* input image. The expected image is 320 x 320, with three channels (red, blue, and green) per pixel"
)

# Set content properties for feature
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (_metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (_metadata_fb.ContentProperties.ImageProperties)

input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (_metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]

input_meta.processUnits = [input_normalization]

input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]

input_meta.stats = input_stats

In [9]:
# Create first output tensor info (locations)
location_meta = _metadata_fb.TensorMetadataT()
location_meta.name = "locations"
location_meta.description = "The locations of the detected boxes."

# Set content properties for bounding box
location_meta.content = _metadata_fb.ContentT()
location_meta.content.contentProperties = _metadata_fb.BoundingBoxPropertiesT()
location_meta.content.contentProperties.index = [1, 0, 3, 2]
location_meta.content.contentProperties.type = _metadata_fb.BoundingBoxType.BOUNDARIES
location_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.BoundingBoxProperties


# Set range
location_meta.content.range = _metadata_fb.ValueRangeT()
location_meta.content.range.min = 2
location_meta.content.range.max = 2

# Index 0: Corresponds to the x-coordinate of the top-left corner (x_min).
# Index 1: Corresponds to the y-coordinate of the top-left corner (y_min).
# Index 2: Corresponds to the x-coordinate of the bottom-right corner (x_max).
# Index 3: Corresponds to the y-coordinate of the bottom-right corner (y_max).


In [10]:
# Create second output tensor info (classes)
classes_meta = _metadata_fb.TensorMetadataT()
classes_meta.name= "classes"
classes_meta.description = "The classes of the detected boxes."

classes_meta.content = _metadata_fb.ContentT()
classes_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
classes_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties

label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(LABELMAP_FILE)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS

classes_meta.associatedFiles = [label_file]

In [11]:
# Create third output tensor info (scores)
scores_meta = _metadata_fb.TensorMetadataT()
scores_meta.name= "scores"
scores_meta.description = "The scores of the detected boxes."

scores_meta.content = _metadata_fb.ContentT()
scores_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
scores_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties

In [12]:
# Create forth output tensor info (number of detections)
num_meta = _metadata_fb.TensorMetadataT()
num_meta.name= "number of detections"
num_meta.description = "The number of the detected boxes."

num_meta.content = _metadata_fb.ContentT()
num_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
num_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties

In [13]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [location_meta, classes_meta, scores_meta, num_meta]
model_meta.subgraphMetadata = [subgraph]

# Initializes a FlatBuffers Builder object with an initial size of 0
builder = flatbuffers.Builder(0)
builder.Finish(model_meta.Pack(builder), _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = builder.Output()

In [14]:
populator = _metadata.MetadataPopulator.with_model_file(OUTPUT_MODEL_PATH)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([LABELMAP_FILE])
populator.populate()

In [15]:
displayer = _metadata.MetadataDisplayer.with_model_file(OUTPUT_MODEL_PATH)

json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(OUTPUT_MODEL_PATH_JSON, "w") as f:
  f.write(json_file)